In [186]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [187]:
cd '/content/drive/MyDrive/capstone'

/content/drive/MyDrive/capstone


In [188]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

In [189]:
df = pd.read_csv('suncare_keyword_score.csv')

In [190]:
# 1.정규화
scaler = StandardScaler()
scaled_values = scaler.fit_transform(df.drop(columns=['Unnamed: 0','link','name']))
normalized_df = pd.DataFrame(scaled_values, columns=df.columns[3:28])
normalized_df['name'] = df['name']
normalized_df['link'] = df['link']
df = normalized_df.copy()
df.columns

Index(['수분', '유분', '보습', '속건조', '자극', '진정', '탄력', '윤기', '트러블', '트러블개선', '미백효과',
       '피부톤개선', '피부결개선', '주름개선', '모공관리', '각질제거', '흡수력', '무게감', '밀림', '흘러내림',
       '미끌거림', '끈적임', '향', '양', 'name', 'link'],
      dtype='object')

In [199]:
# 샘플 요청
sample_request = {
    '수분': 0,
    '유분': 0,
    '보습' : 0,
    '속건조' : 1,
    '자극': 1,
    '진정': -1,
    '탄력': 0,
    '윤기': 0,
    '트러블': 0,
    '트러블개선': 0,
    '미백효과': 0,
    '피부톤개선': 0,
    '피부결개선': 0,
    '주름개선': 0,
    '모공관리': 0,
    '각질제거': 0,
    '흡수력': 0,
    '무게감': 0,
    '밀림': 0,
    '흘러내림': 0,
    '미끌거림': 0,
    '끈적임': 0,
    '향': 0,
    '양': 0

}

In [200]:
#1. 요청에 맞는 아이템 상위 n%의 df를 리턴하는 함수
def items_for_request(sample_request, df):
  request_features = {key: value for key, value in sample_request.items() if value != 0} # request에 나타난 특성
  percentage = 0.2
  num_of_percentage = round(len(df) * percentage) # 상위 percentage
  request_columns = list(request_features.keys()) # request에 나타난 특성의 키워드
  for key, value in request_features.items(): # 키워드의 부호를 각 score에 곱함
    df[key] *= value
  temp = df[(df[request_columns] > 0).all(axis=1)] # 각 키워드에 맞는 상품만 가져옴
  temp['request_score'] = temp[request_columns].prod(axis=1) # 각 점수가 고른 상품을 위해 점수를 곱함
  requested_df = temp.nlargest(num_of_percentage, 'request_score')

  return requested_df

In [201]:
#2. 나머지 특성을 이용해 클러스터링
def clustering(sample_request, requested_df):
  request_features = {key: value for key, value in sample_request.items() if value != 0}
  request_columns = list(request_features.keys())
  temp_without_request = requested_df.drop(columns=request_columns)
  temp = temp_without_request.select_dtypes(exclude=['object']) # temp: 요청 특성을 제외한 데이터프레임
  # 최적 k 찾기
  wcss = []
  max_k = int(len(requested_df) / 1.5)
  for k in range(1, max_k + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(temp)
    wcss.append(kmeans.inertia_)
  # WCSS의 변화량 계산
  wcss_diff = [wcss[i] - wcss[i + 1] for i in range(len(wcss) - 1)]

  #return temp
  threshold = 1 # 변화량의 임계값 설정
  optimal_k = 4 # 최소 추천 개수

  #optimal_k 업데이트
  for i in range(len(wcss_diff)):
    if abs(wcss_diff[i]) < threshold:
        optimal_k = i + 2
        break

  # 클러스터링
  kmeans = KMeans(n_clusters=optimal_k, random_state=42)
  temp['group'] = kmeans.fit_predict(temp)
  temp['name'] = requested_df['name']
  temp['link'] = requested_df['link']

  return temp

In [202]:
rdf = items_for_request(sample_request, df)
rdf.shape

<ipython-input-200-585aa092fc59>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['request_score'] = temp[request_columns].prod(axis=1) # 각 점수가 고른 상품을 위해 점수를 곱함


(9, 27)

In [206]:
cluster_df = clustering(sample_request, rdf)
print(len(cluster_df))

9


In [204]:
max_request_scores = cluster_df.loc[cluster_df.groupby('group')['request_score'].idxmax()]
max_request_scores.drop(columns = ['request_score', 'group'], inplace = True)
print(max_request_scores)

          수분        유분        보습        탄력        윤기       트러블     트러블개선  \
4   0.107711 -0.573770 -0.705634  0.582637 -0.151788  0.616150 -0.042936   
25 -0.416547 -0.273949 -0.582450  0.463013 -0.772415  0.478047  0.035165   
15  0.248626 -0.574472 -0.665093  0.728923  0.217524  0.629734  0.351405   
36 -0.063095 -0.131034 -0.517343  0.438598 -0.369370  0.526887 -0.147078   

        미백효과     피부톤개선     피부결개선  ...       흡수력       무게감        밀림      흘러내림  \
4   0.554103  0.833881  0.698179  ... -0.513554  0.627302  0.667414  0.604798   
25  0.564362  0.164961  0.340535  ... -0.808643  0.453091  0.438197  0.435017   
15  0.692813  0.746192  0.848499  ... -0.313775  0.661585  0.660687  0.682080   
36  0.355586  0.184991  0.254103  ... -0.431175  0.499328  0.586469  0.506025   

        미끌거림       끈적임         향         양  \
4   0.604000  0.591489 -0.082747 -0.251647   
25  0.459921  0.483044  0.757573 -0.302865   
15  0.655109  0.639127  0.282253  0.035023   
36  0.503299  0.530418 -0.442

In [205]:
for index, row in max_request_scores.iterrows():
    product_name = row['name']
    product_link = row['link']
    print(f"상품명: {product_name}, 상품링크: {product_link}")
    print('>>',end = ' ')
    for column in max_request_scores.select_dtypes(exclude=['object']).columns:
        value = row[column]
        if value > 0:
            print(f"{column}↑",end = " ")  # 양수
        elif value < 0:
            print(f"{column}↓",end = " ")  # 음수
        else:
            print(f"{column}-", end = " ")  # 0일 경우
    print()

상품명: 어퓨 퓨어 블록 톤업 선베이스 이엑스 SPF50+ PA+++, 50ml, 1개, 상품링크: https://www.coupang.com/vp/products/1601750342?itemId=2825937523&vendorItemId=3683860523
>> 수분↑ 유분↓ 보습↓ 탄력↑ 윤기↓ 트러블↑ 트러블개선↓ 미백효과↑ 피부톤개선↑ 피부결개선↑ 주름개선↑ 모공관리↑ 각질제거↑ 흡수력↓ 무게감↑ 밀림↑ 흘러내림↑ 미끌거림↑ 끈적임↑ 향↓ 양↓ 
상품명: 토니모리 바이탈 비타12 보송 선스틱 SPF50+ PA++++, 22g, 1개, 상품링크: https://www.coupang.com/vp/products/6565919185?itemId=760331130&vendorItemId=4918164790
>> 수분↓ 유분↓ 보습↓ 탄력↑ 윤기↓ 트러블↑ 트러블개선↑ 미백효과↑ 피부톤개선↑ 피부결개선↑ 주름개선↑ 모공관리↑ 각질제거↑ 흡수력↓ 무게감↑ 밀림↑ 흘러내림↑ 미끌거림↑ 끈적임↑ 향↑ 양↓ 
상품명: 아이오페 UV 쉴드 톤업 선 SPF50+ PA++++, 50ml, 1개, 상품링크: https://www.coupang.com/vp/products/5546686662?itemId=21187243366&vendorItemId=89982098034
>> 수분↑ 유분↓ 보습↓ 탄력↑ 윤기↑ 트러블↑ 트러블개선↑ 미백효과↑ 피부톤개선↑ 피부결개선↑ 주름개선↑ 모공관리↑ 각질제거↑ 흡수력↓ 무게감↑ 밀림↑ 흘러내림↑ 미끌거림↑ 끈적임↑ 향↑ 양↑ 
상품명: 니베아 프로텍트 앤 라이트 필 퍼펙트 선 세럼, 90ml, 1개, 상품링크: https://www.coupang.com/vp/products/7266009200?itemId=18854427614&vendorItemId=90920951405
>> 수분↓ 유분↓ 보습↓ 탄력↑ 윤기↓ 트러블↑ 트러블개선↓ 미백효과↑ 피부톤개선↑ 피부결개선↑ 주름개선↑ 모공관리↑ 각질제거↑ 흡수력↓ 무게감↑ 밀림↑ 흘러내림↑ 미끌거